https://github.com/erhwenkuo/deep-learning-with-keras-notebooks

### 時尚服飾圖像辨識  
圖像分類被廣泛地應用到很多的領域中，從醫學掃描中危及生命的疾病識別到電視卡通角色的識別。  
  
MNIST數據集（可以說）是最常用的圖像分類入門數據集。它包含手寫數字的28x28灰度圖像，每個灰度圖像都有一個關聯的標籤，用於指示圖像上手寫的數字（0到9之間的整數）。  
  
MNIST在過去幾年中越來越受歡迎, 但也引起了一些問題。  
  
使用MNIST來進行圖像辨識的範例有什麼問題呢？  
1. MNIST太簡單了 - 卷積網絡(Convolution network)可以很容易地在MNIST上達到99.7%的辨識力，即使經典的機械學習(Machine learning)算法也能達到97%。
2. MNIST被過度使用 - 幾乎每個有深度學習經驗的人都至少遇到過MNIST一次。
3. MNIST無法有效代表現代的電視視覺相關的任務 - Keras的主要貢獻者(François Chollet)也特別提及這一點。
> François Chollet  
Many good ideas will not work well on MNIST (e.g. batch norm). Inversely many bad ideas may work on MNIST and no transfer to real CV.  
2:51 AM - Apr 14, 2017

### 資料預處理 (Data Preprocessing)
預處理圖片的第一步是調整它們的大小。我們需要有相同大小的所有照片進行訓練。 我會將數據資料轉換型別為float32來節省一些記憶的用量並對它們進行歸一化（除以255）。  
  
然後使用one-hot編碼來將10類別的標籤(label)轉換為向量(vector)：

In [1]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 載入資料
data_train = pd.read_csv('data/fashion-mnist_train.csv')
data_test = pd.read_csv('data/fashion-mnist_test.csv')

num_classes = 10 # 標籤總共有10類
img_rows, img_cols, img_channels = 28, 28, 1 # 圖像是 28像素 x 28像素 (灰階: 1)
input_shape = (img_rows, img_cols, img_channels) # (圖像的height, 圖像的width, 圖像的顏色通道數channel)

X = np.array(data_train.iloc[:, 1:]) # Dataframe中 idx(1 ~ 784)的欄都是像素值

# 進行標籤的one-hot編碼
y = to_categorical(np.array(data_train.iloc[:, 0])) # Dataframe 中 idx(0)的欄是標籤

# 把訓練資料進行拆分成訓練(80%)與驗證(20%)資料集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

# 測試資料的處理
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

# 對向量進行shape的轉換以符合訓練的input要求
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)

# 對每個像素進行型別轉換與歸一化
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255
X_val = X_val.astype('float32')/255

Using TensorFlow backend.
/Users/unahsu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### 網絡模型 (Model)構建  
現在我們來定義我們的模型架構。我們將使用具有6個卷積層的前饋(feed-forward)網絡，然後是完全連接的隱藏層。 我們也將在兩者之間使用Dropout層來防止網絡"過擬合(overfitting)"。

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

def create_model_six_conv(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(num_classes, activation='softmax'))
    
    return model;

在訓練模型之前，我們需要將模型配置為學習算法並進行編譯。我們需要指定: loss: 損失函數，我們要優化。我們不能使用MSE，因為它是不連續的數值。因此，我們使用：categorical_crossentropy optimizer: 我們使用標準隨機梯度下降(Stochastic gradient descent)與涅斯捷羅夫動量(Nesterov momentum) metric: 由於我們正在處理一個分類問題，我們用度量是accuracy。

In [3]:
from keras.optimizers import Adam

batch_size = 256
epochs = 50

#圖像的shape是 (28,28,1)
model = create_model_six_conv((img_rows, img_cols, img_channels)) # 初始化一個模型

# 秀出模型架構
model.summary() 

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
__________

### 訓練 (Training)  
現在，我們的模型已經準備好了。在訓練期間，我們的模型將進行迭代批量訓練，每個次的訓練資料的大小為batch_size。對於每批次，模型將會計算出梯度(gradient)，並自動更新網絡的權重。對所有訓練集的一次迭代被稱為一次的循環(epoch)。訓練通常會一直進行到損失收斂於一個常數。

### 訓練過程的可視化

### 驗證評估 (Evaluation)

### 每一種類別的預測正確率

顯然，我們的分類器對第6類別的圖像辨識在精度和召回方面的表現相對差。也許我們會在可視化正確和錯誤的預測之後獲得更多的洞察力。

for i, incorrect in enumerate(incorrects[0:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(X_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], y_true[incorrect]))
    plt.tight_layout()
    
plt.show()

### 總結 (Conclusion)  
在這篇文章中有一些個人學習到的一些有趣的重點:
1. 深度學習也可以很摩登很現代感
2. 只要有創造力與天馬行空的想法,很多我們眼睛看到的東西都可以變成應用AI的場景
3. 每個公司都可以擁有自己的訓練圖像資料集
4. 電腦宅男也可以跟時尚(fashion)掛上勾了, 下次我們可以說我們是"AI時尚宅男" ...太潮了!!
### 參考:
. CNN with Keras  
. zalandoresearch/fashion-mnist  
. Classifying clothes using Tensorflow  
. Keras官網